In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

Idée: on veut trouver quels types d'acteurs sont choisis pour quels type de rôles et de films et par période. On pourrait séparer les films par genre et périodes de dix ans. Ensuite, pour ces films, séparer les acteurs par characters et voir quels types d'acteurs (sexe, âge, ethnicité, taille) jouent dans quels type de films et quels sont leur rôles. 

On pourrait ensuite s'intéressait à la ressemblence des acteurs au sein d'un groupe, voir s'il y a un trend, genre acteurs qui joent le rôle du protagoniste dans les films d'actions des années 80 comme James Bond sont toujours des hommes dans leur 30-40, grand, bg et blanc. 
On pourrait ensuite utiliser comme autre data le Wikidata où on a plein d'infos en plus sur les acteurs: couleur des yeux, chevelure, barbe ou pas, nombre de partenaires, etc.. et leurs PHOTOS ! 
On pourrait utiliser les photos pour voir s'il y a une ressemblence flagrante entre eux => on est entrain de voir si c'est facilement implémentable dans python et s'il y a déjà un code pour ça sur github. 

Au final, ça serait trop cool de pouvoir implementer notre photo et prédire dans quel type de films on pourrait jouer. Je pense que ça risque d'être trop compliqué et partir dans un projet de Machine Learning et algorythme mais on peut en tout cas utiliser les infos en plus de Wikidata pour déterminer si les acteurs se ressemblent entre eux ou pas. 

Voila